In [89]:
from dataclasses import dataclass
from typing import ClassVar, Any


@dataclass
class Node:
    key: Any
    color: bool
    parent: object
    left: object
    right: object

    def grand_parent(self):
        if(self.parent != None):
            return self.parent.parent
        else:
            return None
    
    def uncle(self):
        gp = self.grand_parent()
        if gp is None:
            return None
        if self.parent == gp.left:
            return gp.right
        else:
            return gp.left




In [92]:
right_l = Node(4, False, None, None, None)
right_r = Node(5, False, None, None, None)
root = Node(1, False, None, None, None)
right = Node(2, False, root, right_l, right_r)
left = Node(3, False, root, None, None)
root.left = left
root.right = right
right_r.parent = right
right_l.parent = left

print(right_l.uncle().key)


2
